In [ ]:
import numpy as np
from random import randint
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# data
train_labels = []   
train_samples = []
for i in range(1000):
    train_labels.append(0)
    train_samples.append(randint(13, 64))
    train_labels.append(1)
    train_samples.append(randint(65, 100))
for i in range(50):
    train_labels.append(1)
    train_samples.append(randint(13, 64))
    train_labels.append(0)
    train_samples.append(randint(65, 100))
    
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)

In [ ]:
# preprocessing
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))

In [ ]:
print(train_samples[:10])
print(scaled_train_samples[:10])

here you need to focus on the format of the data:

there are array, and the first dimension is data point.

In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam, RMSprop 

# from keras.metrics import categorical_crossentropy

In [ ]:
# build model
model = Sequential([
    Dense(16, input_shape=(1,), activation="relu"),
    Dense(32, activation="relu"),
    Dense(2, activation="softmax")
])

# another way
model = Sequential()
model.add(Dense(16, input_shape=(1,), activation = "relu"))
model.add(Dense(2, activation="relu"))
model.add(Dense(2, activation="softmax"))

# third way
x = Input(shape=(1, ))
h = Dense(16)(x)
h = Dense(2)(h)
y = Dense(2)(h)
model = Model(inputs=x, outputs=y)

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(lr=0.001),
              loss='sparse_categorical_crossentropy', 
              metrics=["accuracy"])

In [ ]:
model.fit(x=scaled_train_samples, 
          y=train_labels, 
          batch_size=10, 
          epochs=20, 
          #shuffle=True,
          verbose=2
         )

# verbose: Integer. 0, 1, or 2. Verbosity mode. 
# 0 = silent, 
# 1 = progress bar, 
# 2 = one line per epoch.
# epochs is to be understood as "final epoch". 
# validation_data will override validation_split

In [ ]:
# consider validation
model.fit(x=scaled_train_samples,
          y=train_labels,
          epochs=5,
          validation_split=0.2)

In [ ]:
# validation with valiation data set
model.fit(x=scaled_train_samples,
          y=train_labels,
          epochs=5,
          validation_data=(scaled_train_samples[-100:], train_labels[-100:]))

In [ ]:
model.predict(scaled_train_samples[-100:])

In [ ]:
model.predict_classes(scaled_train_samples[-100:])   # pick the maximum

# save and load model

In [ ]:
import os
PROJECT_FOLDER = os.path.join(os.path.dirname(os.getcwd()), "storage")
os.chdir(PROJECT_FOLDER)
print(os.getcwd())

In [ ]:
from keras.models import load_model

In [ ]:
model.save("dummie_model.h5")
model = load_model("dummie_model.h5")

In [ ]:
# another way, save model shape and weights differently
import json
from keras.models import model_from_json

In [ ]:
# save
json_string = model.to_json()
with open("dummie_model.json", "w") as f:
    json.dump(json_string, f)

model.save_weights("dummie_model_weights.h5")

# ---------------------------------------------
# load
with open("dummie_model.json", "r") as f:
    model = model_from_json(json.load(f))

model.load_weights("dummie_model_weights.h5")

# more examples

## (1) linear model

-- data --

In [ ]:
import matplotlib.pyplot as plt

X = np.linspace(-1, 1, 200)
np.random.shuffle(X)    # randomize the data
Y = 0.5 * X + 2 + np.random.normal(0, 0.05, (200, ))

# plot data
plt.scatter(X, Y)
plt.show()

In [ ]:
X_train, Y_train = X[:160], Y[:160]     # first 160 data points
X_test, Y_test = X[160:], Y[160:]       # last 40 data points

print(X_train.shape)
print(Y_train.shape)

-- model --

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(1,)))

model.summary()

model.compile(optimizer = "sgd", loss= "mse")

In [ ]:
checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5',
                             monitor='val_acc', 
                             save_best_only=True, 
                             mode='auto')

In [ ]:
history = model.fit(X_train, 
                    Y_train, 
                    batch_size=50,  # default to be 32 
                    epochs=3,
                    callbacks=[checkpoint],
                    verbose=0)

print(history.history)
# returns the dictionized thing as model.evaluate
# print(history.history['loss'])

In [ ]:
# get the loss and metrics
model.evaluate(X_test, Y_test, batch_size=10, verbose=0)  

In [ ]:
# get the weights
W,b = model.layers[0].get_weights()
print(W)
print(b)

## (2) NN

-- data --

In [ ]:
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)
print(y_train.shape)

# data pre-processing
# flat the input data
X_train = X_train.reshape(X_train.shape[0], -1) / 255.   # normalize
X_test = X_test.reshape(X_test.shape[0], -1) / 255.      # normalize

# one-hot output data
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print(X_train.shape)
print(y_train.shape)

 -- model --

In [ ]:
model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])
model.summary()

In [ ]:
# Another way to define your optimizer
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# We add metrics to get more results you want to see
model.compile(optimizer=rmsprop, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

print('test loss: ', loss)
print('test accuracy: ', accuracy)